In [ ]:
import os
import glob
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from typing import List


%matplotlib inline
%config IPCompleter.use_jedi = False

In [ ]:
## pandas básico

## Viajes de Ecobici

In [ ]:
df_viajes = pd.read_csv('data/ecobici/2017-06.csv')
print(df_viajes.shape)
df_viajes.head()

In [ ]:
df_viajes.dtypes

## Procesar archivos

In [ ]:

def procesar_datos(archivos: List[str]):
    """Concatenar y limpiar base de datos"""
    dfs = [
        pd.read_csv(path, dtype={'Bici': str})
        for path in archivos
    ]
    df = pd.concat(dfs, axis=0, ignore_index=True)
    # parse dates
    df = df.assign(
        Fecha_Retiro=df.Fecha_Retiro + ' ' + df.Hora_Retiro,
        Fecha_Arribo=df.Fecha_Arribo + ' ' + df.Hora_Arribo,
    ).drop(['Hora_Retiro', 'Hora_Arribo'], axis=1)
    df = df.assign(
        Fecha_Retiro=pd.to_datetime(df.Fecha_Retiro, 
                                    format='%d/%m/%Y %H:%M:%S'),
        Fecha_Arribo=pd.to_datetime(df.Fecha_Arribo,
                                    format='%d/%m/%Y %H:%M:%S'),
    )
    delta = df.Fecha_Arribo - df.Fecha_Retiro
    duracion_viaje = (delta.dt.seconds / 60).round().astype(int)
    df = df.assign(duracion_viaje=duracion_viaje)
    return df

archivos = glob.glob('data/ecobici/*.csv')
print(archivos)
df_viajes = procesar_datos(archivos)
print(df_viajes.shape)
df_viajes.head()

In [ ]:
df_test = df_viajes.assign(mes=df_viajes.Fecha_Retiro.dt.month)
df_test.groupby('mes').Bici.count().plot.bar()

In [ ]:
df_test = df_viajes.assign(mes=df_viajes.Fecha_Retiro.dt.month)
df_test.groupby('mes').Bici.count().plot.bar()

## Archivo parquet

In [ ]:
## guardar como parquet
# df_viajes.to_parquet('data/viajes_ecobici.parquet', engine='pyarrow')

# Leer parquet
df_viajes = pd.read_parquet('data/viajes_ecobici.parquet', engine='pyarrow')
print(df_viajes.shape)
df_viajes.head()

In [ ]:
df_viajes.groupby('Genero_Usuario').Bici.count()

In [ ]:
with plt.style.context('seaborn-talk'):
    df_viajes.Edad_Usuario.plot.hist(bins=50)

In [ ]:
df_viajes.Edad_Usuario.describe()

In [ ]:
df_viajes.groupby('Genero_Usuario').Edad_Usuario.mean()

In [ ]:
with plt.style.context('seaborn-talk'):
    df_viajes.duracion_viaje.plot.hist(bins=50)

In [ ]:
df_viajes.duracion_viaje.describe()

In [ ]:
# Filtrar
df_viajes = df_viajes.loc[df_viajes.duracion_viaje <= 60]
df_viajes = df_viajes.loc[df_viajes.duracion_viaje > 2]
print(df_viajes.shape)
df_viajes.duracion_viaje.describe()

In [ ]:
with plt.style.context('seaborn-talk'):
    df_viajes.duracion_viaje.plot.hist(bins=20)

In [ ]:
df_viajes = df_viajes.assign(dia_semana=df_viajes.Fecha_Retiro.dt.dayofweek)
with plt.style.context('bmh'):
    df_viajes.groupby('dia_semana').Bici.count().plot.bar(figsize=(12, 7))

In [ ]:
df_viajes = df_viajes.assign(hora_dia=df_viajes.Fecha_Retiro.dt.hour)
with plt.style.context('randy_olson_style'):
    df_viajes.groupby('hora_dia').Bici.count().plot.bar(figsize=(12, 7), color='firebrick')

In [ ]:

dias_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
dias_semana = {i: d for i, d in enumerate(dias_semana)}
df_viajes = df_viajes.assign(
    dia_semana=df_viajes.dia_semana.map(dias_semana)
)
df_viajes.head()

In [ ]:
# df_viajes.dia_semana.value_counts(dropna=False)

In [ ]:
# heatmap

with plt.style.context('seaborn-talk'):
    
    df_table = (df_viajes.groupby(['dia_semana', 'hora_dia'], as_index=False)
                         .Bici.count()
                         .pivot(index='dia_semana',
                                columns='hora_dia',
                                values='Bici'))
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20, 10))
    sns.heatmap(df_table.reindex(dias_semana.values()), cmap='inferno', annot=True, fmt='d', ax=ax)
    

In [ ]:

with plt.style.context('seaborn-talk'):
    gp = df_viajes.groupby('')

In [ ]:
df_serie_tiempo = df_viajes.set_index('Fecha_Retiro')
df_serie_tiempo.head()

In [ ]:
## Slice index

In [ ]:
with plt.style.context('randy_olson_style'):
    df_ts = df_serie_tiempo.resample('D').Bici.count()
    ts_mean = df_ts.rolling(10, center=True).mean()
    ts_std = df_ts.rolling(10, center=True).std()
    ax = df_ts.plot()
    ax = ts_mean.plot()
    plt.fill_between(df_ts.index,
                     ts_mean - (1 * ts_std),
                     ts_mean + (1 * ts_std),
                     color='g', alpha=0.2)

In [ ]:
## Ejercicio
## heamap de retiro-arribo


# mask = np.zeros_like(df_aux, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

## Conseguir Lat y Lons de estaciones

In [ ]:
def pedir_info_estaciones(client_id: str, client_secret: str):
    """Pide la información de las estaciones de ecobici."""
    data = {'client_id': client_id,
            'client_secret': client_secret,
            'grant_type': 'client_credentials'}
    # pedir token
    response = requests.get(
        url='https://pubsbapi.smartbike.com/oauth/v2/token',
        params=data)
    if response.status_code != 200:
        raise IOError(
            'No se pudo conseguir token. '
            'Respuesta diferente de 200'
        )
    acces_token = response.json()['access_token']
    # pedir datos de estaciones
    response = requests.get(
        url='https://pubsbapi.smartbike.com/api/v1/stations.json',
        params={'access_token': acces_token}
    )
    if response.status_code != 200:
        raise IOError(
            'No se pudo conseguir info de estaciones. '
            'Respuesta diferente de 200'
        )
    estaciones_json = response.json()['stations']
    return estaciones_json


client_id = os.environ.get('ECOBICI_CLIENT_ID')
client_secret = os.environ.get('ECOBICI_CLIENT_SECRET')

estaciones_json = pedir_info_estaciones(client_id, client_secret)
df_estaciones = pd.io.json.json_normalize(estaciones_json)
df_estaciones.head()

In [ ]:
# df_estaciones = (df_estaciones.rename(columns={'location.lat': 'latitud',
#                                               'location.lon': 'longitud',
#                                               'id': 'estacion_id'})
#                               .loc[:, ['estacion_id', 'latitud', 'longitud']])
# df_estaciones.to_csv('data/estaciones_ecobici.csv', index=False, encoding='utf-8', quoting=1)
# df_estaciones.head()

In [ ]:
df_estaciones = pd.read_csv('data/estaciones_ecobici.csv')
print(df_estaciones.shape)
df_estaciones.head()

In [ ]:
with plt.style.context('seaborn-talk'):
    df_estaciones.plot.scatter(x='longitud', y='latitud')

In [ ]:
from IPython.display import Image
Image('SQL.png')

In [ ]:

columnas = ['Ciclo_Estacion_Retiro', 'Fecha_Retiro', 'Ciclo_Estacion_Arribo', 'Fecha_Arribo', 'duracion_viaje']

df_viajes = df_viajes.loc[(df_viajes.duracion_viaje >= 2) & (df_viajes.duracion_viaje < 60)]
df_viajes = df_viajes.loc[:, columnas]
df_viajes.head()

In [ ]:
df_viajes_coords = pd.merge(df_viajes,
                            df_estaciones,
                            how='inner',
                            left_on='Ciclo_Estacion_Retiro',
                            right_on='estacion_id')
df_viajes_coords = df_viajes_coords.drop('estacion_id', axis=1)
df_viajes_coords = df_viajes_coords.rename(columns={'latitud': 'latitud_retiro',
                                                    'longitud': 'longitud_retiro'})
df_viajes_coords = pd.merge(df_viajes_coords,
                            df_estaciones,
                            how='inner',
                            left_on='Ciclo_Estacion_Arribo',
                            right_on='estacion_id')

df_viajes_coords = df_viajes_coords.rename(columns={'latitud': 'latitud_arribo',
                                                    'longitud': 'longitud_arribo'})

df_viajes_coords.head()

## Buenas prácticas

In [ ]:
# funciona pero es muy lento (3m 48s)
def calcular_dist_apply(row):
    delta_lon = row.longitud_retiro - row.longitud_arribo
    delta_lat = row.latitud_retiro - row.latitud_arribo
    return sqrt(delta_lon ** 2 + delta_lat ** 2)

distancia = df_viajes_coords.apply(calcular_dist_apply, axis=1)

In [ ]:
distancia.describe()

In [ ]:

delta_lat = df_viajes_coords.latitud_retiro - df_viajes_coords.latitud_arribo
delta_lon = df_viajes_coords.longitud_retiro - df_viajes_coords.longitud_arribo
distancia = delta_lat ** 2 + delta_lon ** 2
distancia = np.sqrt(distancia)

In [ ]:
distancia.describe()

In [ ]:

def calcular_distancias(df):
    df = df.copy()
    delta_lat = df_viajes_coords.latitud_retiro - df_viajes_coords.latitud_arribo
    delta_lon = df_viajes_coords.longitud_retiro - df_viajes_coords.longitud_arribo
    distancia = delta_lat ** 2 + delta_lon ** 2
    distancia = np.sqrt(distancia)
    return distancia

distancia = calcular_distancias(df_viajes_coords)